# 感情分析をRNNでやってみます。

In [1]:
import numpy as np

In [2]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf

In [3]:
with open('./reviews.txt', 'r') as f:
    reviews = f.read()

In [4]:
#reviews

In [5]:
reviews[:200]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  '

In [6]:
with open('./labels.txt','r') as f:
    labels = f.read()

In [7]:
labels[:200]

'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npo'

In [8]:
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [9]:
all_text[:200]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s '

In [10]:
# reviews[:200]

In [11]:
# words[:200]

In [12]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)

In [13]:
vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}

In [14]:
reviews_int = []
for each in reviews:
    reviews_int.append([vocab_to_int[word] for word in each.split()])

In [15]:
# reviews_int レビューを数字の配列に変換する。

In [16]:
# ラベルをベクトルにする

In [17]:
labels = labels.split('\n')
labels = np.array([1 if each =='positive' else 0 for each in labels])

In [18]:
review_lens = Counter([len(x) for x in reviews_int])

In [19]:
review_lens[0]

1

In [20]:
max(review_lens)

2514

In [21]:
non_zero_idx = [ii for ii, review in enumerate(reviews_int) if len(review) != 0]

In [22]:
len(non_zero_idx)

25000

In [23]:
reviews_int[-1]

[]

In [24]:
reviews_int = [reviews_int[ii] for ii in non_zero_idx]

In [25]:
labels = np.array([labels[ii] for ii in non_zero_idx])

In [26]:
seq_len = 200
features = np.zeros((len(reviews_int), seq_len), dtype=int)

In [27]:
for i, row in enumerate(reviews_int):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [28]:
# features[:10,:100]

In [29]:
# データをトレーニング用と検証用に分割する

In [30]:
split_frac = 0.8

In [31]:
split_idx = int(len(features)*0.8)

In [32]:
train_x, val_x = features[:split_idx], features[split_idx:]

In [33]:
train_y, val_y = labels[:split_idx], labels[split_idx:]

In [34]:
test_idx = int(len(val_x)*0.5)

In [35]:
val_x, test_x = val_x[:test_idx], val_x[test_idx:]

In [36]:
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

In [37]:
print("Train set: \t\t{}".format(train_x.shape))

Train set: 		(20000, 200)


In [38]:
print("Validation set: \t{}".format(val_x.shape))

Validation set: 	(2500, 200)


In [39]:
print("Test set: \t\t{}".format(test_x.shape))

Test set: 		(2500, 200)


## グラフの定義

In [40]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001

In [41]:
n_words = len(vocab_to_int) + 1

graph = tf.Graph()
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32,[None,None], name='inputs')
    labels_ = tf.placeholder(tf.int32,[None,None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')

In [42]:
embed_size = 300

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words,embed_size),-1,1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### LSTMセルとレイヤーを定義する

In [43]:
with graph.as_default():
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)

AttributeError: module 'tensorflow_core.compat.v1' has no attribute 'contrib'

In [ ]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell,embed,initial_state=initial_state)

### 推定値の計算と損失関数、最適化処理の定義

In [ ]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:,-1],1,activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_,predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 学習精度の計測

In [ ]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### バッチ（指定長のデータ）を返すモジュール定義

In [ ]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

## トレーニング（学習）

In [ ]:
epochs = 10
with graph.as_default():
    saver = tf.train.Saver()
    
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        for ii, (x,y) in enumerate(get_batches(train_x,train_y, batch_size),1):
            feed = {inputs_: x,
                   labels_: y[:,None],
                   keep_prob: 0.5,
                   initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                     "Iteration: {}".format(iteration),
                     "Training Loss: {:.3f}".format(loss))
                
            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x,y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                           labels_: y[:,None],
                           keep_prob: 1,
                           initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state],feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Value Acc: {:.3f}".format(np.mean(val_acc)))
            iteration += 1
    saver.save(sess, "checkpoint/sentiment.ckpt")
        
        
        
        
        
                
            

In [ ]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoint'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x,y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
               labels_: y[:,None],
               keep_prob: 1,
               initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test Accuracy: {:.3f}".format(np.mean(test_acc)))